Компьютерное зрение на датасете Horses Or Humans Dataset

In [ ]:
# установим нужные библиотеки
!pip install tensorflow
!pip install prettytable
!pip install openpyxl

Импортируем библиотеки

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras.callbacks import Callback
from keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import layers
from keras.layers.experimental import preprocessing
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D, Flatten, Dense
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import load_model
from keras.optimizers import SGD, Adam
from keras.applications import MobileNetV2
from PIL import Image
from prettytable import PrettyTable, ALL


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import os
import warnings
warnings.filterwarnings('ignore')
import shutil

Возьмем готовый датасет с kaggle - Horses Or Humans Dataset

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# создадим папки для хранения данных
os.makedirs('Data', exist_ok=True)
os.makedirs('Data/Train/Horses', exist_ok=True)
os.makedirs('Data/Train/Humans', exist_ok=True)
os.makedirs('Data/Test/Horses', exist_ok=True)
os.makedirs('Data/Test/Humans', exist_ok=True)

base_path = os.getcwd()
horse_counter = 0
human_counter = 0

for i, dataset in enumerate(tfds.load('horses_or_humans', split=['train', 'test'])):
    if i==0: #для тренировочной выборки
        set_path = os.path.join(base_path, 'Data/Train')
    else: #для тестовой выборки
        set_path = os.path.join(base_path, 'Data/Test')

    for row in list(dataset):
        im = Image.fromarray(row['image'].numpy())
        if row['label'] == 0: #0 для horse and 1 для human
            class_path = os.path.join(set_path, 'Horses')
            file_path = os.path.join(class_path, "horse_{}.jpeg".format(horse_counter))
            horse_counter += 1
        elif row['label'] == 1: #0 для horse and 1 для human
            class_path = os.path.join(set_path, 'Humans')
            file_path = os.path.join(class_path, "human_{}.jpeg".format(horse_counter))
            human_counter += 1
        im.save(file_path)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteHEYDHL/horses_or_humans-train.tfrecord*..…

Generating test examples...:   0%|          | 0/256 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteHEYDHL/horses_or_humans-test.tfrecord*...…

Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [ ]:
# выведем данные о количестве элементов в выборках по классам
print('Number of Horse Images in the Training Set:', len(os.listdir('Data/Train/Horses')))
print('Number of Human Images in the Training Set:', len(os.listdir('Data/Train/Humans')))
print('\n')
print('Number of Horse Images in the Testing Set:', len(os.listdir('Data/Test/Horses')))
print('Number of Human Images in the Testing Set:', len(os.listdir('Data/Test/Humans')))

Number of Horse Images in the Training Set: 500
Number of Human Images in the Training Set: 259


Number of Horse Images in the Testing Set: 128
Number of Human Images in the Testing Set: 66


In [ ]:
# аугментируем наши данные
print('Training Set:')
train_gen = ImageDataGenerator(
    rescale=(1./255),
    rotation_range=0.4,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

train_generator = train_gen.flow_from_directory(
    'Data/Train',
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
)


print('Testing Set:')
test_gen = ImageDataGenerator(rescale=(1./255))

test_generator = test_gen.flow_from_directory(
    'Data/Test',
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
)

Training Set:
Found 759 images belonging to 2 classes.
Testing Set:
Found 194 images belonging to 2 classes.


In [ ]:
# создадим нашу модель

model = Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPool2D((2,2)),
    Conv2D(32, (3,3), activation='relu'),
    MaxPool2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D((2,2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# выведем информацию по нашей модели
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 16)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 64)        0

In [ ]:
# скомпилируем модель
model.compile(loss='binary_crossentropy',
              metrics=['accuracy'])

# обучим модель
history = model.fit(
    train_generator,
    epochs=15,
)

Epoch 1/15
24/24 [==============================] - 24s 948ms/step - loss: 0.4393 - accuracy: 0.8116
Epoch 2/15
24/24 [==============================] - 20s 834ms/step - loss: 0.3484 - accuracy: 0.8498
Epoch 3/15
24/24 [==============================] - 22s 885ms/step - loss: 0.3295 - accuracy: 0.8709
Epoch 4/15
24/24 [==============================] - 20s 802ms/step - loss: 0.2753 - accuracy: 0.8893
Epoch 5/15
24/24 [==============================] - 21s 875ms/step - loss: 0.1880 - accuracy: 0.9223
Epoch 6/15
24/24 [==============================] - 19s 798ms/step - loss: 0.2630 - accuracy: 0.9051
Epoch 7/15
24/24 [==============================] - 21s 821ms/step - loss: 0.2289 - accuracy: 0.9104
Epoch 8/15
24/24 [==============================] - 24s 995ms/step - loss: 0.2097 - accuracy: 0.9065
Epoch 9/15
24/24 [==============================] - 21s 881ms/step - loss: 0.1683 - accuracy: 0.9420
Epoch 10/15
24/24 [==============================] - 27s 1s/step - loss: 0.1402 - accuracy:

In [ ]:
# выведем результаты по тестовой выборке
test_acc = model.evaluate(test_generator, verbose=0)[1]
print('Accuracy модели на тестовой выборке: ', round(test_acc,3))

Accuracy модели на тестовой выборке:  0.593


0_эксперимент: изменим параметры: изменим epoch

In [ ]:
# изменим epoch

# скомпилируем модель
model.compile(loss='binary_crossentropy',
              metrics=['accuracy'])


# обучим модель
history = model.fit(
    train_generator,
    epochs=5
)

# выведем результаты по тестовой выборке
test_acc_0 = model.evaluate(test_generator, verbose=0)[1]
print('Accuracy модели на тестовой выборке при epoch = 5: ', round(test_acc_0,3))

Epoch 1/5
24/24 [==============================] - 22s 869ms/step - loss: 0.1686 - accuracy: 0.9473
Epoch 2/5
24/24 [==============================] - 21s 817ms/step - loss: 0.0675 - accuracy: 0.9697
Epoch 3/5
24/24 [==============================] - 21s 866ms/step - loss: 0.0817 - accuracy: 0.9710
Epoch 4/5
24/24 [==============================] - 21s 816ms/step - loss: 0.0759 - accuracy: 0.9736
Epoch 5/5
24/24 [==============================] - 27s 1s/step - loss: 0.0688 - accuracy: 0.9802
Accuracy модели на тестовой выборке при epoch = 5:  0.541


Изменение количества эпох с 15 до 5 ухудшило  accuracy модели до 0.541

1_эксперимент: изменим параметры: добавим оптимизатор: SGD

In [ ]:
# добавим оптимизатор: SGD

# скомпилируем модель
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=0.001, momentum=0.99),
              metrics=['accuracy'])


# обучим модель
history = model.fit(
    train_generator,
    epochs=15
)

# выведем результаты по тестовой выборке
test_acc_1 = model.evaluate(test_generator, verbose=0)[1]
print('Accuracy модели на тестовой выборке с оптимизатором SGD: ', round(test_acc_1,3))

Epoch 1/15
24/24 [==============================] - 21s 848ms/step - loss: 0.0822 - accuracy: 0.9684
Epoch 2/15
24/24 [==============================] - 23s 913ms/step - loss: 0.3592 - accuracy: 0.8498
Epoch 3/15
24/24 [==============================] - 27s 1s/step - loss: 0.3600 - accuracy: 0.8551
Epoch 4/15
24/24 [==============================] - 21s 829ms/step - loss: 0.1934 - accuracy: 0.9302
Epoch 5/15
24/24 [==============================] - 19s 775ms/step - loss: 0.1625 - accuracy: 0.9473
Epoch 6/15
24/24 [==============================] - 24s 981ms/step - loss: 0.1922 - accuracy: 0.9302
Epoch 7/15
24/24 [==============================] - 26s 1s/step - loss: 0.1979 - accuracy: 0.9341
Epoch 8/15
24/24 [==============================] - 19s 787ms/step - loss: 0.1916 - accuracy: 0.9302
Epoch 9/15
24/24 [==============================] - 21s 848ms/step - loss: 0.1436 - accuracy: 0.9433
Epoch 10/15
24/24 [==============================] - 20s 824ms/step - loss: 0.1219 - accuracy: 0.

Добавление оптимизатора SGD незначительно улучшило accuracy модели до 0.603

2_эксперимент: изменим параметры: изменим learning_rate

In [ ]:
# изменим learning_rate

# скомпилируем модель
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=0.01, momentum=0.99),
              metrics=['accuracy'])

# обучим модель
history = model.fit(
    train_generator,
    epochs=15
)

# выведем результаты по тестовой выборке
test_acc_2 = model.evaluate(test_generator, verbose=0)[1]
print('Accuracy модели на тестовой выборке с оптимизатором SGD и learning rate = 0.01:', round(test_acc_2,3))

Epoch 1/15
24/24 [==============================] - 22s 876ms/step - loss: 0.3616 - accuracy: 0.9078
Epoch 2/15
24/24 [==============================] - 22s 906ms/step - loss: 0.5793 - accuracy: 0.7615
Epoch 3/15
24/24 [==============================] - 23s 941ms/step - loss: 0.6225 - accuracy: 0.7154
Epoch 4/15
24/24 [==============================] - 19s 774ms/step - loss: 0.6447 - accuracy: 0.6588
Epoch 5/15
24/24 [==============================] - 20s 805ms/step - loss: 0.6462 - accuracy: 0.6588
Epoch 6/15
24/24 [==============================] - 18s 746ms/step - loss: 0.6504 - accuracy: 0.6588
Epoch 7/15
24/24 [==============================] - 20s 829ms/step - loss: 0.6433 - accuracy: 0.6588
Epoch 8/15
24/24 [==============================] - 26s 1s/step - loss: 0.6524 - accuracy: 0.6588
Epoch 9/15
24/24 [==============================] - 21s 887ms/step - loss: 0.6601 - accuracy: 0.6588
Epoch 10/15
24/24 [==============================] - 19s 765ms/step - loss: 0.6479 - accuracy:

Accuracy модели на тестовой выборке с оптимизатором SGD и learning rate = 0.01 улучшило accuracy модели до 0.66

3_эксперимент: изменим параметры: изменим оптимизатор, learning_rate, количество epoch

In [ ]:
# изменим оптимизатор на Adam

# скомпилируем модель
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.1),
              metrics=['accuracy'])

# обучим модель
history = model.fit(
    train_generator,
    epochs=20
)

# выведем результаты по тестовой выборке
test_acc_3 = model.evaluate(test_generator, verbose=0)[1]
print('Accuracy модели на тестовой выборке с оптимизатором Adam, learning rate = 0.1 и количеством эпох 20:', round(test_acc_3,3))


Epoch 1/20
24/24 [==============================] - 23s 914ms/step - loss: 0.6612 - accuracy: 0.6403
Epoch 2/20
24/24 [==============================] - 23s 938ms/step - loss: 0.6448 - accuracy: 0.6588
Epoch 3/20
24/24 [==============================] - 24s 1s/step - loss: 0.6476 - accuracy: 0.6588
Epoch 4/20
24/24 [==============================] - 24s 979ms/step - loss: 0.6501 - accuracy: 0.6588
Epoch 5/20
24/24 [==============================] - 22s 914ms/step - loss: 0.6460 - accuracy: 0.6588
Epoch 6/20
24/24 [==============================] - 24s 982ms/step - loss: 0.6440 - accuracy: 0.6588
Epoch 7/20
24/24 [==============================] - 29s 1s/step - loss: 0.6423 - accuracy: 0.6588
Epoch 8/20
24/24 [==============================] - 19s 797ms/step - loss: 0.6430 - accuracy: 0.6588
Epoch 9/20
24/24 [==============================] - 22s 911ms/step - loss: 0.6438 - accuracy: 0.6588
Epoch 10/20
24/24 [==============================] - 20s 801ms/step - loss: 0.6458 - accuracy: 0.

In [ ]:
Model_Table = PrettyTable()
Model_Table.hrules=ALL
Model_Table.field_names = (["Базовый при epoch = 15: ", "При epoch = 5: ", "с SGD: ", "с SGD и lr = 0.01:", "с Adam, lr = 0.1, при epoch = 20"])
Model_Table.add_row([ round(test_acc,3), round(test_acc_0,3), round(test_acc_1,3), round(test_acc_2,3), round(test_acc_3,3) ])
print("Accuracy по результатам экспериментов")
print(Model_Table)

Accuracy по результатам экспериментов
+--------------------------+-----------------+---------+--------------------+----------------------------------+
| Базовый при epoch = 15:  | При epoch = 5:  | с SGD:  | с SGD и lr = 0.01: | с Adam, lr = 0.1, при epoch = 20 |
+--------------------------+-----------------+---------+--------------------+----------------------------------+
|          0.593           |      0.541      |  0.603  |        0.66        |               0.66               |
+--------------------------+-----------------+---------+--------------------+----------------------------------+


При помощи ручного перебора параметров, мы повысили accuracy нашей модели с 0.593 до 0.66

In [ ]:
# загрузим базовую модель MobileNetV2 для transfer learning

base_model = MobileNetV2(
            input_shape=(150,150,3),
            include_top=False,
            weights='imagenet')

# заморозим веса модели
for layer in base_model.layers:
    layer.trainable = False

# добавим несколько слоев в модель
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(1, activation='sigmoid')(x)


new_model = tf.keras.Model(base_model.input, x)

new_model.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# обучим новую модель
history = new_model.fit(
    train_generator,
    epochs=20
)

9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/20
24/24 [==============================] - 31s 1s/step - loss: 0.4140 - accuracy: 0.9657
Epoch 2/20
24/24 [==============================] - 24s 976ms/step - loss: 0.5310 - accuracy: 0.9881
Epoch 3/20
24/24 [==============================] - 22s 899ms/step - loss: 0.1114 - accuracy: 0.9960
Epoch 4/20
24/24 [==============================] - 26s 1s/step - loss: 0.0482 - accuracy: 0.9974
Epoch 5/20
24/24 [==============================] - 24s 988ms/step - loss: 0.0608 - accuracy: 0.9974
Epoch 6/20
24/24 [==============================] - 24s 944ms/step - loss: 0.0756 - accuracy: 0.9974
Epoch 7/20
24/24 [==============================] - 25s 1s/step - loss: 0.0496 - accuracy: 0.9974
Epoch 8/20
24/24 [==============================] - 23s 908ms/step - loss: 0.1554 - accuracy: 0.9960
Epoch 9/20
24/24 [==============================] - 23s 935ms/step - loss: 0.4766 - accuracy: 0.9947
Epoch 10/20
24/24 [==================

In [ ]:
# выведем результаты по тестовой выборке
test_acc_4 = model.evaluate(test_generator, verbose=0)[1]
print('Accuracy модели на тестовой выборке с  transfer learning :', round(test_acc_4,3))

Accuracy модели на тестовой выборке с  transfer learning : 0.66


Результат по нашей модели c transfer learning стоит на одном уровне с оптимизаторами adam и sgd

По результатам экспериментов, повышение accuracy модели весомее всего произошло благодаря изменению оптимизаторов